# 개발환경(開發環境) setting

In [ ]:
import random
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.layers import LSTM, Dropout
import datetime
import os
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.font_manager as fm
from numpy import array
import matplotlib as mpl
from IPython.core.display import display, HTML
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from statsmodels.tsa.seasonal import seasonal_decompose, STL
import urllib.request
import requests
import json
import seaborn as sns
from prophet import Prophet 
from prophet.plot import add_changepoints_to_plot
from fbprophet.plot import plot_plotly, plot_components_plotly
from fbprophet.plot import add_changepoints_to_plot
import holidays
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.rcParams['font.family'] = 'AppleGothic'

In [ ]:
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
계절 = pd.read_csv('데이터/계절.csv')
preprocessing = pd.read_csv('데이터/preprocessing.csv')
submission = pd.read_csv('데이터/sample_submission.csv')

# This is EDA and Pre-processing

In [ ]:
# 계절 아래 2행 추가
for a in range(2):
    new_row = []
    new_row.append('가을')
    계절 = 계절.append(pd.Series(new_row, index=계절.columns), ignore_index=True)

In [ ]:
# 현재 없는 2020년 11월 5일을 추가해보자
new_row = []
new_row.append('2020-11-05')
new_row.append('목요일')
for t in range(10):
    new_row.append( (preprocessing.iloc[1769, t + 2] + preprocessing.iloc[1770, t + 2]) / 2 )
new_row.append(11)
new_row.append(2020)
new_row.append(3)

preprocessing = preprocessing.append(pd.Series(new_row, index=preprocessing.columns), ignore_index=True)

temp = preprocessing.loc[1770].copy()
preprocessing.loc[1770] = preprocessing.loc[1771]
preprocessing.loc[1771] = temp
preprocessing

In [ ]:
# 1주 전 가격들의 평균내기(월~일, 0으로 된 날 개수 세는 코드와 그에 따른 평균 내는 코드로)
# index 기준 10~16, 17~23, ...
# index 기준 0~9는 걍 그 날 가격으로 합니다.
# this is 1772row version.
for 변수 in ['대파', '팽이버섯', '깻잎', '캠벨얼리', '토마토']:
    A = [];B = [];a = (len(preprocessing) - 10) // 7;b = (len(preprocessing) - 10) % 7
    for x in range(10):
        A.append(preprocessing[변수 + '_거래량(kg)'].iloc[x])
        B.append(preprocessing[변수 + '_가격(원/kg)'].iloc[x])
    for y in range(a):
        s = 0;t = 0
        for z in range(7):
            s += preprocessing[변수 + '_거래량(kg)'].iloc[(y * 7) + z + 3]
            t += preprocessing[변수 + '_가격(원/kg)'].iloc[(y * 7) + z + 3]
        v_1 = (preprocessing[변수 + '_거래량(kg)'].iloc[(y * 7) + 3:(y * 7) + 10] == 0).sum()
        v_2 = (preprocessing[변수 + '_가격(원/kg)'].iloc[(y * 7) + 3:(y * 7) + 10] == 0).sum()
        for w in range(7):
            A.append(s / (7 - v_1))
            B.append(t / (7 - v_2))
    for z in range(7):
        s += preprocessing[변수 + '_거래량(kg)'].iloc[1760 + z]
        t += preprocessing[변수 + '_가격(원/kg)'].iloc[1760 + z]
    v_1 = (preprocessing[변수 + '_거래량(kg)'].iloc[1760:1767] == 0).sum()
    v_2 = (preprocessing[변수 + '_가격(원/kg)'].iloc[1760:1767] == 0).sum()
    for w in range(b):
        A.append(s / (7 - v_1))
        B.append(t / (7 - v_2))
    Adf = pd.DataFrame(A, columns = [변수 + '_거래량의_1주전_평균'])
    Bdf = pd.DataFrame(B, columns = [변수 + '_가격의_1주전_평균'])

    # 2주 전 가격들의 평균내기(월~일, 0으로 된 날 개수 세는 코드와 그에 따른 평균 내는 코드로)
    # index 기준 10~23, 24~37, ...
    # index 기준 0~16는 걍 그 날 가격으로 합니다.
    C = [];D = []
    for x in range(17):
        C.append(preprocessing[변수 + '_거래량(kg)'].iloc[x])
        D.append(preprocessing[변수 + '_가격(원/kg)'].iloc[x])
    for y in range(a - 1):
        p = 0;q = 0
        for z in range(14):
            p += preprocessing[변수 + '_거래량(kg)'].iloc[(y * 7) + z + 3]
            q += preprocessing[변수 + '_가격(원/kg)'].iloc[(y * 7) + z + 3]
        v_1 = (preprocessing[변수 + '_거래량(kg)'].iloc[(y * 7) + 3:(y * 7) + 17] == 0).sum()
        v_2 = (preprocessing[변수 + '_가격(원/kg)'].iloc[(y * 7) + 3:(y * 7) + 17] == 0).sum()
        for w in range(7):
            C.append(p / (14 - v_1))
            D.append(q / (14 - v_2))
    for z in range(14):
        p += preprocessing[변수 + '_거래량(kg)'].iloc[1753 + z]
        q += preprocessing[변수 + '_가격(원/kg)'].iloc[1753 + z]
    v_1 = (preprocessing[변수 + '_거래량(kg)'].iloc[1753:1767] == 0).sum()
    v_2 = (preprocessing[변수 + '_가격(원/kg)'].iloc[1753:1767] == 0).sum()
    for w in range(b):
        C.append(p / (14 - v_1))
        D.append(q / (14 - v_2))
    Cdf = pd.DataFrame(C, columns = [변수 + '_거래량의_2주전_평균'])
    Ddf = pd.DataFrame(D, columns = [변수 + '_가격의_2주전_평균'])

    preprocessing = pd.concat([preprocessing, Adf, Bdf, Cdf, Ddf, 계절], axis=1)

농산물_for_Model = preprocessing.copy()
# 농산물_for_LSTM_LGBM = pd.get_dummies(preprocessing, columns=["요일"])

# This is Modeling

### 나는 3week도 넣고 싶었(고 코드 또한 몇 줄만 손 보면 바로 나오)지만 팀원들에게 짤막

### LightGBM

In [1]:
# Year Dream School 2기(期) 교육생(敎育生) 최고은, 장한결이 구현(具顯)한 code cell이 여기부터 시작된다.
class Nong1:
    
    def __init__(self, df, test):
        self.df = df
        self.test = test
        self.df = pd.concat([self.df, self.test], axis=0).reset_index(drop=True)
        self.df = self.df[1:].replace(0, np.NaN)
        self.df = self.df.interpolate().fillna(0)
        self.df.date = pd.to_datetime(self.df.date)
        self.df = pd.concat([self.df, pd.get_dummies(self.df['요일'])], axis=1)
        self.feature = self.df.columns[2:]
        self.df = self.df.reset_index(drop=True)

    def set_feature(self,name):
        self.name = name
        self.name1 = name+ "_가격(원/kg)"
        self.name2 = name+ "_거래량(kg)"
        self.name3 = name+ "_거래량의_1주전_평균"
        self.name4 = name+ '_가격의_1주전_평균'
        self.name5 = name+ '_거래량의_2주전_평균'
        self.name6 = name+ '_가격의_2주전_평균'
        self.name7 = '계절'
        self.feature = [self.name1, self.name2, self.name3, self.name4, self.name5, self.name6, self.name7, '금요일',   '목요일',   '수요일',   '월요일',   '일요일',   '토요일',   '화요일', 'resid']

    def set_target(self,week):
        if week == 1:
            self.df['target'] = self.df[self.name1].shift(-8)
        elif week == 2:
            self.df['target'] = self.df[self.name1].shift(-15)
        elif week == 3:
            self.df['target'] = self.df[self.name1].shift(-22)
        elif week == 4:
            self.df['target'] = self.df[self.name1].shift(-29)

        self.df['resid'] = 0
        stl = STL(self.df[['date', self.name1]].set_index('date'), period=12)
        res = stl.fit()
        self.df['resid'] = res.resid.values
        
    def set_model_lgb(self):
        import lightgbm as lgb
        self.scaler = RobustScaler()
        self.df[self.feature] = self.scaler.fit_transform(self.df[self.feature])
        self.df_learn = self.df[self.df['target'].notnull()]
        self.X = self.df_learn[self.feature]
        self.y = self.df_learn['target']

        self.train_split = 1733
        self.valid_split = -1

        self.X_train = self.X[:self.train_split]
        self.y_train = self.y[:self.train_split]
        self.X_valid = self.X[self.train_split:self.valid_split]
        self.y_valid = self.y[self.train_split:self.valid_split]
        self.X_test = self.X[self.valid_split:]
        self.y_test = self.y[self.valid_split:]

        self.train_ds = lgb.Dataset(self.X_train,  label = self.y_train)
        self.valid_df = lgb.Dataset(self.X_valid,  label = self.y_valid)

        self.params = {'learning_rate': 0.01, 
              'max_depth': 6, 
              'boosting': 'gbdt', 
              'objective': 'regression', 
              'metric': 'mse', 
              'is_training_metric': True, 
              'num_leaves': 100, 
              'feature_fraction': 0.8, 
              'bagging_fraction': 0.8, 
              'bagging_freq': 5, 
              'num_threads':8
                 }
        self.num_round = 1000

        self.model_lgb = lgb.train(self.params, self.train_ds, self.num_round, self.valid_df)

    def get_plot_lgb(self):  
    #   print(self.X_test)
        self.y_pred = self.model_lgb.predict(self.X_test)
    #   print(self.X_test.shape, self.y_pred.shape)
        self.RMSE = mean_squared_error(self.y_test, self.y_pred) ** 0.5
        print(self.RMSE.round(2))
        plt.figure(figsize=(20, 10), dpi=300)
        plt.title(self.name + ' 가격 예측 결과'+   '   RMSE : ' + str(self.RMSE)[:7])
        plt.ylabel(self.name + ' 가격')
        plt.plot(np.array(self.y_train), alpha = 0.9, label = 'Real')
        plt.plot(self.model_lgb.predict(self.X_train), alpha = 0.6, linestyle = "--", label = 'Predict')
        plt.legend()
        plt.show()
    

    def get_price_lgb(self):
        # self.X_test = X_test[self.feature]
        self.price = self.model_lgb.predict(self.X_test)
        return self.price[0]

In [ ]:
my_nong1 = Nong1(농산물_for_Model, df_1106)

# '대파', '깻잎', '팽이버섯', '토마토', '캠벨얼리'

my_nong1.set_feature('캠벨얼리')

my_nong1.set_target(1)

In [ ]:
my_nong1.set_model_lgb()

In [ ]:
my_nong1.get_plot_lgb()

In [ ]:
my_nong1.y_pred, my_nong1.y_test

In [ ]:
# from sympy import Shi

weeks = [1, 2, 3, 4]
features = ['대파', '깻잎', '팽이버섯', '토마토', '캠벨얼리'] 

week1=[]
week2=[]
week3=[]
week4=[]

for week in weeks:
    print(week)
    for feature in features:
        my_nong1 = Nong1(농산물_for_Model, df_1106)
        my_nong1.set_feature(feature)
        my_nong1.set_target(week)
        print(my_nong1.set_target(week))
        my_nong1.set_model_lgb()
        if week == 1:
            week1.append(my_nong1.get_price_lgb())
        if week == 2:
            week2.append(my_nong1.get_price_lgb())
        if week == 3:
            week3.append(my_nong1.get_price_lgb())
        if week == 4:
            week4.append(my_nong1.get_price_lgb())
        print(feature)

In [ ]:
week1, week2, week3, week4

In [ ]:
from pyrsistent import b

date = '20201106'

feature = [ '대파_가격(원/kg)', '깻잎_가격(원/kg)','팽이버섯_가격(원/kg)', '토마토_가격(원/kg)', '캠벨얼리_가격(원/kg)']
submission = submission.set_index('예측대상일자')
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+1week', feature] = week1
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+2week', feature] = week2
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+3week', feature] = week3
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+4week', feature] = week4
submission.reset_index(drop=False, inplace=True)
title = 'submissionLGBM' + date + '.csv'

In [ ]:
submission.drop(columns=['배추_가격(원/kg)', '무_가격(원/kg)', '양파_가격(원/kg)', '건고추_가격(원/kg)',
       '마늘_가격(원/kg)',  '얼갈이배추_가격(원/kg)', '양배추_가격(원/kg)',
       '시금치_가격(원/kg)', '미나리_가격(원/kg)', '당근_가격(원/kg)', '파프리카_가격(원/kg)', '새송이_가격(원/kg)', 
       '청상추_가격(원/kg)', '백다다기_가격(원/kg)', '애호박_가격(원/kg)','샤인마스캇_가격(원/kg)'], inplace= True)

In [ ]:
submission2 = submission.round(2)
submission2 = submission2[3:]
submission2

In [ ]:
# 실제값과 비교!
private2 = pd.read_csv(raw_path+'private_data.csv')
private2 = private2[['date','요일','대파_가격(원/kg)', '깻잎_가격(원/kg)','팽이버섯_가격(원/kg)', '토마토_가격(원/kg)', '캠벨얼리_가격(원/kg)']]
private2_week = private2[::7]
private2_week[0:5]

### LSTM

In [ ]:
class Nong1:
    

    def __init__(self, df, test):
        self.df = df
        self.test = test
        self.df = pd.concat([self.df, self.test], axis=0).reset_index(drop=True)
        self.df = self.df[1:].replace(0, np.NaN)
        self.df = self.df.interpolate().fillna(0)
        self.df.date = pd.to_datetime(self.df.date)
        self.df = pd.concat([self.df, pd.get_dummies(self.df['요일'])], axis=1)
        self.feature = self.df.columns[2:]
        self.df = self.df.reset_index(drop=True)

    def set_feature(self,name):
        self.name = name
        self.name1 = name+ "_가격(원/kg)"
        self.name2 = name+ "_거래량(kg)"
        self.name3 = name+ "_거래량의_1주전_평균"
        self.name4 = name+ '_가격의_1주전_평균'
        self.name5 = name+ '_거래량의_2주전_평균'
        self.name6 = name+ '_가격의_2주전_평균'
        self.name7 = '계절'
        self.feature = [self.name1, self.name2, self.name3, self.name4, self.name5, self.name6, self.name7, '금요일',   '목요일',   '수요일',   '월요일',   '일요일',   '토요일',   '화요일', 'resid']

    def set_target(self,week):
        if week == 1:
            self.df['target'] = self.df[self.name1].shift(-8)
        elif week == 2:
            self.df['target'] = self.df[self.name1].shift(-15)
        elif week == 3:
            self.df['target'] = self.df[self.name1].shift(-22)
        elif week == 4:
            self.df['target'] = self.df[self.name1].shift(-29)

        self.df['resid'] = 0
        stl = STL(self.df[['date', self.name1]].set_index('date'), period=12)
        res = stl.fit()
        self.df['resid'] = res.resid.values
        
    def set_model_lstm(self):
        import tensorflow as tf
        import keras
        from tensorflow.keras import Sequential
        from tensorflow.keras import layers
        from tensorflow.keras.layers import Dense, LSTM, Dropout
        from keras.callbacks import ModelCheckpoint, EarlyStopping
        from statsmodels.tsa.seasonal import seasonal_decompose, STL

        self.scaler = RobustScaler()
        self.df[self.feature] = self.scaler.fit_transform(self.df[self.feature])
        self.df_learn = self.df[self.df['target'].notnull()]
        self.X = self.df_learn[self.feature]
        self.y = self.df_learn['target']

        self.train_split = -1

        self.X_train = self.X[:self.train_split].values.reshape(-1,1,len(self.feature))
        self.y_train = self.y[:self.train_split].values.reshape(-1,1,1)
        self.X_test = self.X[self.train_split:].values.reshape(-1,1,len(self.feature))
        self.y_test = self.y[self.train_split:].values.reshape(-1,1,1)

        self.model = Sequential()
        self.model.add(layers.Activation('relu'))
        self.model.add(tf.compat.v1.keras.layers.LSTM(100, input_shape=(21,len(self.feature)), return_sequences=True))
        self.model.add(Dropout(0.1))
        self.model.add(layers.Dense(30))
        self.model.add(Dropout(0.1))
        self.model.add(layers.Dense(1))
        self.model.compile(optimizer='adam', loss='mse')
        self.early_stopping = EarlyStopping(patience=30)

        self.model.fit(self.X_train, self.y_train, epochs=1000, batch_size=32, validation_split = 0.1, callbacks=[self.early_stopping], verbose=0)

    def get_plot_lstm(self):  
        self.y_pred = self.model.predict(self.X_test)
        print(self.X_test.shape, self.y_pred.shape)
        self.RMSE = mean_squared_error(self.y_test.reshape(-1,1), self.y_pred.reshape(-1,1)) ** 0.5

        plt.figure(figsize=(20, 10), dpi=300)
        plt.title(self.name + ' 가격 예측 결과'+   '   RMSE : ' + str(self.RMSE)[:7])
        plt.ylabel(self.name + ' 가격')
        plt.plot(np.array(self.y_train.reshape(-1,1)), alpha = 0.9, label = 'Real')
        plt.plot(self.model.predict(self.X_train).reshape(-1,1), alpha = 0.6, linestyle = "--", label = 'Predict')
        plt.legend()
        plt.show()
        print(self.RMSE.round(2))

    def get_price_lstm(self):
        # self.X_test = X_test[self.feature]
        self.price = self.model.predict(self.X_test.reshape(-1,1,len(self.feature)))
        return self.price[0][0][0]

In [ ]:
my_nong1 = Nong1(농산물_for_Model, df_1106)
# '대파', '깻잎', '팽이버섯', '토마토', '캠벨얼리'
my_nong1.set_feature('깻잎')

my_nong1.set_target(1)

In [ ]:
my_nong1.set_model_lstm()

In [ ]:
my_nong1.get_plot_lstm()

In [ ]:
my_nong1.y_pred, my_nong1.y_test

In [ ]:
# from sympy import Shi

weeks = [1, 2, 3, 4]
features = ['대파', '깻잎', '팽이버섯', '토마토', '캠벨얼리'] 

week1=[]
week2=[]
week3=[]
week4=[]

for week in weeks:
    print(week)
    for feature in features:
        my_nong1 = Nong1(농산물_for_Model, df_1106)
        my_nong1.set_feature(feature)
        my_nong1.set_target(week)
        print(my_nong1.set_target(week))
        my_nong1.set_model_lstm()
        if week == 1:
            week1.append(my_nong1.get_price_lstm())
        if week == 2:
            week2.append(my_nong1.get_price_lstm())
        if week == 3:
            week3.append(my_nong1.get_price_lstm())
        if week == 4:
            week4.append(my_nong1.get_price_lstm())
        print(feature)

In [ ]:
week1, week2, week3, week4

In [ ]:
from pyrsistent import b

date = '20201106'

feature = [ '대파_가격(원/kg)', '깻잎_가격(원/kg)','팽이버섯_가격(원/kg)', '토마토_가격(원/kg)', '캠벨얼리_가격(원/kg)']
submission = submission.set_index('예측대상일자')
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+1week', feature] = week1
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+2week', feature] = week2
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+3week', feature] = week3
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+4week', feature] = week4
submission.reset_index(drop=False, inplace=True)
title = 'submissionLSTM' + date + '.csv'

In [ ]:
submission.drop(columns=['배추_가격(원/kg)', '무_가격(원/kg)', '양파_가격(원/kg)', '건고추_가격(원/kg)',
       '마늘_가격(원/kg)',  '얼갈이배추_가격(원/kg)', '양배추_가격(원/kg)',
       '시금치_가격(원/kg)', '미나리_가격(원/kg)', '당근_가격(원/kg)', '파프리카_가격(원/kg)', '새송이_가격(원/kg)', 
       '청상추_가격(원/kg)', '백다다기_가격(원/kg)', '애호박_가격(원/kg)','샤인마스캇_가격(원/kg)'], inplace= True)

In [ ]:
submission2 = submission.round(2)
submission2 = submission2[3:]
submission2

In [ ]:
# 실제값과 비교!
private2 = pd.read_csv(raw_path+'private_data.csv')
private2 = private2[['date','요일','대파_가격(원/kg)', '깻잎_가격(원/kg)','팽이버섯_가격(원/kg)', '토마토_가격(원/kg)', '캠벨얼리_가격(원/kg)']]
private2_week = private2[::7]
private2_week[0:5]

### Prophet

In [ ]:
!pip install holiday
!pip install prophet
!pip install pystan~=2.14
!pip install fbprophet 
!pip install statsmodels==0.11.1|

In [ ]:
import matplotlib.font_manager as fm
sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")

In [ ]:
private2 = 농산물_for_Model.copy()
private2 = private2[['date','대파_거래량(kg)', '대파_가격(원/kg)', '깻잎_거래량(kg)', '깻잎_가격(원/kg)',
               '팽이버섯_거래량(kg)', '팽이버섯_가격(원/kg)', '토마토_거래량(kg)', '토마토_가격(원/kg)', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)',]]

In [ ]:
private2['date'] = private2.date.astype(str).str.replace('-','')
private2['date'] = pd.to_datetime(private2.date, format='%Y%m%d')


t = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
private2['요일'] = 0
temp = []
for i in range(len(private2)):
      temp.append(t[int(private2['date'][i].weekday())])
private2['요일'] = temp

private2 = private2[['date', '요일','대파_거래량(kg)', '대파_가격(원/kg)', '깻잎_거래량(kg)', '깻잎_가격(원/kg)',
               '팽이버섯_거래량(kg)', '팽이버섯_가격(원/kg)', '토마토_거래량(kg)', '토마토_가격(원/kg)', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)',]]

In [ ]:
class Nong1:
    
    def __init__(self, df, test):
        self.df = df
        self.test = test
        self.df = pd.concat([self.df, self.test], axis=0).reset_index(drop=True)
        self.df = self.df[1:].replace(0, np.NaN)
        self.df = self.df.interpolate().fillna(0)
        self.df.date = pd.to_datetime(self.df.date)
        self.df = pd.concat([self.df, pd.get_dummies(self.df['요일'])], axis=1)
        self.feature = self.df.columns[2:]
        self.df = self.df.reset_index(drop=True)

    def set_feature(self,name):
        self.name = name
        self.name1 = name+ "_가격(원/kg)"
        self.name2 = name+ "_거래량(kg)"
        self.name3 = name+ "_거래량의_1주전_평균"
        self.name4 = name+ '_가격의_1주전_평균'
        self.name5 = name+ '_거래량의_2주전_평균'
        self.name6 = name+ '_가격의_2주전_평균'
        self.name7 = '계절'
        self.feature = [self.name1, self.name2, self.name3, self.name4, self.name5, self.name6, self.name7, '금요일',   '목요일',   '수요일',   '월요일',   '일요일',   '토요일',   '화요일', 'resid']

    def set_target(self,week):
        if week == 1:
            self.df['target'] = self.df[self.name1].shift(-8)
        elif week == 2:
            self.df['target'] = self.df[self.name1].shift(-15)
        elif week == 3:
            self.df['target'] = self.df[self.name1].shift(-22)
        elif week == 4:
            self.df['target'] = self.df[self.name1].shift(-29)

        self.df['resid'] = 0
        stl = STL(self.df[['date', self.name1]].set_index('date'), period=12)
        res = stl.fit()
        self.df['resid'] = res.resid.values

    def set_model_prophet(self):
        self.scaler = RobustScaler()
        self.df[self.feature] = self.scaler.fit_transform(self.df[self.feature])
        self.df_learn = self.df[self.df['target'].notnull()]
        self.X = self.df_learn[self.feature]
        self.y = self.df_learn['target']

        self.split = 1770
        self.split_end = 2052
        
        # 파라미터
        self.main_df = pd.DataFrame()
        self.main_df['ds'] = self.df['date']
        self.main_df['y'] = self.y
        self.main_trian_df = self.main_df[:self.split]
        self.main_test_df = self.main_df[self.split:self.split_end].reset_index(drop=True)
        self.X_train_prophet_df = self.X[:self.split]
        self.X_test_prophet_df = self.X[self.split:self.split_end].reset_index(drop=True)

        self.yx_train_prophet_df = pd.concat([self.main_trian_df, self.X_train_prophet_df], axis = 1)
        self.yx_test_prophet_df = pd.concat([self.main_test_df, self.X_test_prophet_df], axis = 1).reset_index(drop=True)

        self.prophet = Prophet(growth='linear', #linear -> 추세를 말함
                        # Seasonality
                        seasonality_mode='multiplicative', # 계절성 모델: 'additive' or 'multiplicative'
                        seasonality_prior_scale=10.0, # 계절성 추정 민감도로 높을수록 민감
                        yearly_seasonality='auto', # 연계절성
                        weekly_seasonality='auto', # 주계절성
                        daily_seasonality='auto', #일계절성

                        # Holiday
                        holidays=None, # 휴일 또는 이벤트 시점 dataframe
                        holidays_prior_scale=10.0, # 휴일 추정 민감도로 높을수록 민감

                        # Others
                        interval_width=0.8, # 추세 예측 정확도 구간범위
                        mcmc_samples=0)
        self.prophet.add_country_holidays(country_name='KR') #미리 입력된 한국 휴일 불러오기
        self.prophet.fit(self.yx_train_prophet_df)

    def get_plot_prophet(self):  
        self.future_data = self.prophet.make_future_dataframe(periods = 282, freq = 'd')
        self.forecast_data = self.prophet.predict(self.future_data)
        self.forecast_data = self.forecast_data[['ds','yhat']]
        self.y_pred = self.forecast_data.yhat.values[1770:2052] # -282
        self.rmse = round(math.sqrt(mean_squared_error(self.test_y, self.y_pred)),2)
        print(self.rmse)

        plt.figure(figsize=(20, 10), dpi=300)
        self.pred_fbprophet_y = self.forecast_data.yhat.values[1770:2052]
        self.test_y = self.yx_test_prophet_df.y.values
        plt.plot(self.pred_fbprophet_y, color = 'gold')
        plt.plot(self.test_y, color = 'green')
        plt.legend(['pred', 'test_y'])
        plt.show()

    def get_price_prophet(self):
        self.future_data = self.prophet.make_future_dataframe(periods = 282, freq = 'd')
        self.forecast_data = self.prophet.predict(self.future_data)
        self.forecast_data = self.forecast_data[['ds','yhat']]
        self.y_pred = self.forecast_data.yhat.values[1770:2052]
        self.test_y = self.yx_test_prophet_df.y.values
        self.rmse = round(math.sqrt(mean_squared_error(self.test_y, self.y_pred)),2)
        print(self.rmse)
        self.price = pd.DataFrame()
        self.price = self.forecast_data[['yhat']][1770:2052]
        self.temp_df = pd.DataFrame()
        self.temp_df = self.yx_test_prophet_df
        self.price['real'] = self.yx_test_prophet_df.y.values
        self.temp_df.reset_index(inplace = True)
        self.price['time'] = self.df['date'][1770:2052]
        self.price.dropna(inplace = True)
        self.price.set_index('time', drop=True, inplace=True)
        return self.price

In [ ]:
my_nong1 = Nong1(df, private2)

my_nong1.set_feature('토마토')

my_nong1.set_target(1)

In [ ]:
my_nong1.set_model_prophet()

my_nong1.get_price_prophet()

In [ ]:
my_nong1.get_plot_prophet()

In [ ]:
from sympy import Shi

weeks = [1, 2, 3, 4]
features = ['팽이버섯', '캠벨얼리', '대파', '깻잎', '토마토']  

week1=[]
week2=[]
week3=[]
week4=[]

for week in weeks:
    print(week)
        for feature in features:
        my_nong1 = Nong1(df, private2)
        my_nong1.set_feature(feature)
        my_nong1.set_target(week)
        print(my_nong1.set_target(week))
        my_nong1.set_model_prophet()
        if week == 1:
            week1.append(my_nong1.get_price_prophet())
        if week == 2:
            week2.append(my_nong1.get_price_prophet())
        if week == 3:
            week3.append(my_nong1.get_price_prophet())
        if week == 4:
            week4.append(my_nong1.get_price_prophet())
        print(feature)

In [ ]:
from pyrsistent import b
from datetime import datetime

date = '20201106'

feature = [ '대파_가격(원/kg)', '깻잎_가격(원/kg)','팽이버섯_가격(원/kg)', '토마토_가격(원/kg)', '캠벨얼리_가격(원/kg)']
submission = submission.set_index('예측대상일자')
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+1week', feature] = week1
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+2week', feature] = week2
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+3week', feature] = week3
submission.loc[datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")+'+4week', feature] = week4
submission.reset_index(drop=False, inplace=True)
title = 'submissionProphet' + date + '.csv'

In [ ]:
submission.drop(columns=['배추_가격(원/kg)', '무_가격(원/kg)', '양파_가격(원/kg)', '건고추_가격(원/kg)',
       '마늘_가격(원/kg)',  '얼갈이배추_가격(원/kg)', '양배추_가격(원/kg)',
       '시금치_가격(원/kg)', '미나리_가격(원/kg)', '당근_가격(원/kg)', '파프리카_가격(원/kg)', '새송이_가격(원/kg)', 
       '청상추_가격(원/kg)', '백다다기_가격(원/kg)', '애호박_가격(원/kg)','샤인마스캇_가격(원/kg)'], inplace= True)

In [ ]:
submission2 = submission[228:].round(2)
submission2

In [ ]:
# 실제값과 비교! 
private2 = private2[['date','요일','대파_가격(원/kg)', '깻잎_가격(원/kg)','팽이버섯_가격(원/kg)', '토마토_가격(원/kg)', '캠벨얼리_가격(원/kg)']]
private2_week = private2[::7]
private2_week[1:5]
# Year Dream School 2기(期) 교육생(敎育生) 최고은, 장한결이 구현(具顯)한 code cell이 여기에서 종료된다.

# 이 아래는 담당자 첫 미팅(10/14) 전까지 신나게 하다가 집어치운 것들이지만 명백한 가능성!

### 기상청 털기

In [ ]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math
# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
import chromedriver_autoinstaller
# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm
# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
import random
from time import sleep
import time
# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
N = int(input('지역에 해당하는 정수 stn을 입력하시오 :'))
평균기온 = [];최고기온 = [];최저기온 = [];평균운량 = [];일강수량 = [];날짜 = []
# 크롬 옵션
options = webdriver.ChromeOptions()
# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=1600,1200") # window-size -> 기본 : 1920,1080
# 기상청을 키자
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(1)
for year in range(2016, 2021):
    if year != 2020:
        for month in range(1, 13):
            driver.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn={}&yy={}&mm={}&ob'.format(N, year, month))
            time.sleep(4)
            for u in [1, 3, 5, 7, 9, 11]:
                for v in range(1, 8):
                    try:
                        x = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span'.format(u, v))[0].text
                        if x != ' ':
                            a = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(1)'.format(u+1, v))[0].text
                            b = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(4)'.format(u+1, v))[0].text
                            c = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(7)'.format(u+1, v))[0].text
                            d = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(10)'.format(u+1, v))[0].text
                            e = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(13)'.format(u+1, v))[0].text

                            if len(b) > 5:
                                최고기온.append(float(b[5:].replace('℃', '')))
                            else:
                                최고기온.append((최고기온[-2] + 최고기온[-1]) / 2)

                            if len(c) > 5:
                                최저기온.append(float(c[5:].replace('℃', '')))
                            else:
                                최저기온.append((최저기온[-2] + 최저기온[-1]) / 2)

                            if len(a) > 5:
                                평균기온.append(float(a[5:].replace('℃', '')))
                            elif (len(b) > 5) & (len(c) > 5):
                                평균기온.append((float(b[5:].replace('℃', '')) + float(c[5:].replace('℃', ''))) / 2)
                            else:
                                평균기온.append((평균기온[-2] + 평균기온[-1]) / 2)

                            평균운량.append(d[5:])
                            일강수량.append(e[5:])
                            날짜.append(str(year) + '년' + ' ' + str(month) + '월' + ' ' + x)
                    except:
                        pass
            time.sleep(random.randint(6, 11))
    else:
        for month in range(1, 12):
            driver.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn={}&yy={}&mm={}&ob'.format(N, year, month))
            time.sleep(4)
            for u in [1, 3, 5, 7, 9, 11]:
                for v in range(1, 8):
                    try:
                        x = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span'.format(u, v))[0].text
                        if x != ' ':
                            a = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(1)'.format(u+1, v))[0].text
                            b = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(4)'.format(u+1, v))[0].text
                            c = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(7)'.format(u+1, v))[0].text
                            d = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(10)'.format(u+1, v))[0].text
                            e = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(13)'.format(u+1, v))[0].text

                            if len(b) > 5:
                                최고기온.append(float(b[5:].replace('℃', '')))
                            else:
                                최고기온.append((최고기온[-2] + 최고기온[-1]) / 2)

                            if len(c) > 5:
                                최저기온.append(float(c[5:].replace('℃', '')))
                            else:
                                최저기온.append((최저기온[-2] + 최저기온[-1]) / 2)

                            if len(a) > 5:
                                평균기온.append(float(a[5:].replace('℃', '')))
                            elif (len(b) > 5) & (len(c) > 5):
                                평균기온.append((float(b[5:].replace('℃', '')) + float(c[5:].replace('℃', ''))) / 2)
                            else:
                                평균기온.append((평균기온[-2] + 평균기온[-1]) / 2)

                            평균운량.append(d[5:])
                            일강수량.append(e[5:])
                            날짜.append(str(year) + '년' + ' ' + str(month) + '월' + ' ' + x)
                    except:
                        pass
            time.sleep(random.randint(6, 11))

In [ ]:
테스트 = pd.DataFrame({'날짜':날짜,
                    '평균기온':평균기온,
                    '최고기온':최고기온,
                    '최저기온':최저기온,
                    '평균운량':평균운량,
                    '일강수량':일강수량})

날씨데이터 = 테스트[:-25]

# 평균운량에서 str로 된 실수값을 float화, ' -'이런 놈은 0으로 치환
A = []
for i in 날씨데이터['평균운량']:
    if i == ' -':
        A.append(0)
    else:
        A.append(float(i))
        
# 일강수량에서 ' -'이런 놈은 0으로 치환, str에 mm붙은 건 적절한 슬라이싱
B = []
for j in 날씨데이터['일강수량']:
    if j == ' -':
        B.append(0)
    else:
        B.append(float(j[:-2]))

날씨데이터['평균운량'] = A
날씨데이터['일강수량'] = B
날씨데이터['일교차'] = 날씨데이터['최고기온'] - 날씨데이터['최저기온']

날씨데이터.to_csv("어느지역_날씨.csv", encoding='utf-8-sig')

### 새로운 데이터가 월별로 주어진다면 1772row 제작법

In [ ]:
A = []
for x in range(59):
    if (x == 1) | (x == 49):
        for i in range(29):
            A.append(프레임이름['열이름'].iloc[x])
    elif x in [13, 25, 37]:
        for i in range(28):
            A.append(프레임이름['열이름'].iloc[x])
    elif x in [3,5,8,10, 15,17,20,22, 27,29,32,34, 39,41,44,46, 51,53,56]:
        for i in range(30):
            A.append(프레임이름['열이름'].iloc[x])
    elif x == 58:
        for i in range(6):
            A.append(프레임이름['열이름'].iloc[x])
    else:
        for i in range(31):
            A.append(프레임이름['열이름'].iloc[x])

### 새로운 데이터가 연간으로 주어진다면 1772row 제작법

In [ ]:
A = []
for x in range(5):
    if x == 0:
        for i in range(366):
            A.append(프레임이름['열이름'].iloc[x])
    elif x < 4:
        for i in range(365):
            A.append(프레임이름['열이름'].iloc[x])
    else:
        for i in range(273):
            A.append(프레임이름['열이름'].iloc[x])

### 재배 비율에 맞는 평균 내기

In [ ]:
# 부산 33.6, 창원 17.5, 춘천 17, 경주 12.6, 광주광역시 11, 장수 8.3
final = 중간단계.copy()
final['fin평균기온'] = 중간단계['부산_평균기온'] * (336/1000) + 중간단계['창원_평균기온'] * (175/1000) + 중간단계['춘천_평균기온'] * (17/100) + 중간단계['경주_평균기온'] * (126/1000) + 중간단계['광주광역시_평균기온'] * (11/100) + 중간단계['장수_평균기온'] * (83/1000)
final['fin최고기온'] = 중간단계['부산_최고기온'] * (336/1000) + 중간단계['창원_최고기온'] * (175/1000) + 중간단계['춘천_최고기온'] * (17/100) + 중간단계['경주_최고기온'] * (126/1000) + 중간단계['광주광역시_최고기온'] * (11/100) + 중간단계['장수_최고기온'] * (83/1000)
final['fin최저기온'] = 중간단계['부산_최저기온'] * (336/1000) + 중간단계['창원_최저기온'] * (175/1000) + 중간단계['춘천_최저기온'] * (17/100) + 중간단계['경주_최저기온'] * (126/1000) + 중간단계['광주광역시_최저기온'] * (11/100) + 중간단계['장수_최저기온'] * (83/1000)
final['fin평균운량'] = 중간단계['부산_평균운량'] * (336/1000) + 중간단계['창원_평균운량'] * (175/1000) + 중간단계['춘천_평균운량'] * (17/100) + 중간단계['경주_평균운량'] * (126/1000) + 중간단계['광주광역시_평균운량'] * (11/100) + 중간단계['장수_평균운량'] * (83/1000)
final['fin일강수량'] = 중간단계['부산_일강수량'] * (336/1000) + 중간단계['창원_일강수량'] * (175/1000) + 중간단계['춘천_일강수량'] * (17/100) + 중간단계['경주_일강수량'] * (126/1000) + 중간단계['광주광역시_일강수량'] * (11/100) + 중간단계['장수_일강수량'] * (83/1000)
final['fin일교차'] = final['fin최고기온'] - final['fin최저기온']
final = final.drop(final.columns[9:45], axis=1)